In [ ]:
%pylab inline

import os
import pandas as pd
import pickle
pj = os.path.join

In [ ]:
INPUT_DIR = "/home/floris/DB.Montreal/ArcTrace_Omega/201807_Simon"

In [ ]:
files = os.listdir(INPUT_DIR)
prefixes = []
suffixes = ["_parameters.json","_captured.pickle27","_trials.txt"]
for f in files:
    for s in suffixes:
        if f.endswith(s):
            prefixes.append(f[:-len(s)])
prefixes=list(set(prefixes))
prefixes.sort()

In [ ]:
# Let's take a look at how many trials are in each file ---
for p in prefixes:
    f = pd.read_csv(pj(INPUT_DIR,p+"_trials.txt"),sep=' ')
    print(p,f.shape[0])

Now from the options above, please select your input files, in the right order too.

In [ ]:
loadfiles = """
Pierrick_arc_24_07.16h16m05
Pierrick_arc_24_07.16h22m36
Pierrick_arc_24_07.16h28m29
Pierrick_arc_24_07.16h35m53
""".split()

In [ ]:
alltrials = []
alltrajectories = {}
timestamps = []

cumul_trial = 0
for blocki,f in enumerate(loadfiles):
    tab = pd.read_csv(pj(INPUT_DIR,f+"_trials.txt"),sep=' ')
    tab["block"]=blocki # the index of the block
    
    tab["cumul.trial"]=cumul_trial+tab["trial"]
    cumul_trial=max(tab["cumul.trial"])
    
    alltrials.append(tab)
    
    trial_to_cumul = dict(zip(tab["trial"],tab["cumul.trial"])) # which trial maps to which cumulative trial
    
    trajectories = pickle.load(open(pj(INPUT_DIR,f+"_captured.pickle27"),'rb'))
    
    # Now let's read the trajectories
    for traj in trajectories:
        tr = trial_to_cumul[traj["trial"]]
        alltrajectories[tr]=traj["trajectory"]
        timestamps.append({"cumul.trial":tr,"timestamp":traj["capture.t"]})
    
alltrials = pd.merge(pd.concat(alltrials),pd.DataFrame(timestamps))

# Make a cumulative trial numbering
#alltrials["cumul.trial"]=range(alltrials.shape[0])

In [ ]:
alltrials.head()